In [3]:
import tkinter as tk
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
import joblib
import pandas as pd
from datetime import date

# Load the pre-trained model and transformers
hybrid_model = joblib.load('hybrid_model.sav')
scaler = MinMaxScaler()
pca = PCA(n_components=4)

# Create a function to perform the prediction
def predict_close_price():
    # Read user inputs
    open_price = float(entry_open.get())
    high_price = float(entry_high.get())
    low_price = float(entry_low.get())
    volume = float(entry_volume.get())
    headline_text = entry_headline.get()

    # Fill in the 'Date' column with today's date
    today = date.today().strftime("%Y-%m-d")

    # Calculate Sentiment Score
    sentiment_score = calculate_sentiment_score(headline_text)

    # One-hot encode sentiment
    sentiment_encoded = one_hot_encode_sentiment(sentiment_score)

    # Create a DataFrame with the user input data
    user_data = pd.DataFrame({
        'Date': [today],
        'Open': [open_price],
        'High': [high_price],
        'Low': [low_price],
        'Volume': [volume],
        'Sentiment_Score': [sentiment_score],
        'Sentiment_Negative': [sentiment_encoded[0]],
        'Sentiment_Neutral': [sentiment_encoded[1]],
        'Sentiment_Positive': [sentiment_encoded[2]]
    })

    # Try to read the existing combined_df CSV
    try:
        combined_df = pd.read_csv('combined_df.csv')
    except FileNotFoundError:
        # If the file does not exist, create a new DataFrame
        combined_df = pd.DataFrame()

    # Concatenate the user data with the existing combined_df
    combined_df = pd.concat([combined_df, user_data], ignore_index=True)

    # Save the updated data to the CSV file
    combined_df.to_csv('combined_df.csv', index=False)

    # Prepare the data for prediction
    numerical_features = combined_df[['Open', 'High', 'Low', 'Volume']]
    numerical_features = scaler.fit_transform(numerical_features)
    sentiment_features = combined_df[['Sentiment_Negative', 'Sentiment_Neutral', 'Sentiment_Positive']]

    # Convert numerical features to a DataFrame
    numerical_features_df = pd.DataFrame(numerical_features, columns=['Open', 'High', 'Low', 'Volume'])

    # Combine numerical and sentiment features
    features = pd.concat([numerical_features_df, sentiment_features], axis=1)

    # Apply PCA
    features = pca.fit_transform(features)

    # Predict the close price for the last row (the user input)
    predicted_close = hybrid_model.predict(features[-1].reshape(1, -1))

    # Display the predicted close price
    result_label.config(text=f"Predicted Close Price: {predicted_close[0]:.6f}")



def calculate_sentiment_score(text):
    # Calculate the sentiment score of the headline text
    analysis = TextBlob(text)
    sentiment_score = analysis.sentiment.polarity
    return sentiment_score

def one_hot_encode_sentiment(score):
    # Classify the sentiment score into Negative, Neutral, or Positive
    negative_threshold = -0.2
    positive_threshold = 0.2

    if score < negative_threshold:
        sentiment_encoded = [1, 0, 0]  # Negative sentiment
    elif score > positive_threshold:
        sentiment_encoded = [0, 0, 1]  # Positive sentiment
    else:
        sentiment_encoded = [0, 1, 0]  # Neutral sentiment

    return sentiment_encoded

window = tk.Tk()
# Create input fields with labels and configure them to expand horizontally
tk.Label(window, text="Open Price").pack()
entry_open = tk.Entry(window)
entry_open.pack(fill=tk.X, padx=10)

tk.Label(window, text="High Price").pack()
entry_high = tk.Entry(window)
entry_high.pack(fill=tk.X, padx=10)

tk.Label(window, text="Low Price").pack()
entry_low = tk.Entry(window)
entry_low.pack(fill=tk.X, padx=10)

tk.Label(window, text="Volume").pack()
entry_volume = tk.Entry(window)
entry_volume.pack(fill=tk.X, padx=10)

tk.Label(window, text="Headline Text").pack()
entry_headline = tk.Entry(window)
entry_headline.pack(fill=tk.X, padx=10)

# Create the prediction button
predict_button = tk.Button(window, text="Predict Close Price", command=predict_close_price)
predict_button.pack(pady=10)

# Create a label to display the result
result_label = tk.Label(window, text="")
result_label.pack()

# Start the tkinter application
window.mainloop()
